In [1]:
cd ../../../../

/Users/mykhailoslukvin/repo/dv-data-pipeline


/Users/mykhailoslukvin/.local/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import asyncio
import pandas as pd
from pprint import pprint as print
from tqdm.asyncio import tqdm_asyncio

from dfpp.geo_utils import get_iso3_to_official_name_map
from dfpp.transformation.source_notebooks.ilo_org.retrieve import (
    get_codebook,
    get_indicator,
    list_indicators,
)
from dfpp.transformation.source_notebooks.ilo_org.transform import (
    sanitize_categories,
    transform_indicator,
    SOURCE_NAME
)
from dfpp.publishing.publish import publish_series

MAX_CONCURRENCY = 4

In [3]:
ISO_3_MAP = await get_iso3_to_official_name_map()

In [4]:
df_classif1, df_classif2 = get_codebook()

In [5]:
df_classif1, df_classif2 = sanitize_categories(df_classif1, df_classif2)

In [6]:
indicator_list = list_indicators()
df_indicators = pd.DataFrame(indicator_list)
df_annual_indicators = df_indicators[df_indicators.freq == "A"]
assert (
    df_annual_indicators.id.value_counts().max() == 1
), "Each indicator must have one record"

In [7]:
async def process_indicator(semaphore, indicator, df_classif1, df_classif2, ISO_3_MAP):
    try:
        indicator_id = indicator["id"]

        async with semaphore:
            data = await get_indicator(indicator["id"])
            await asyncio.sleep(5)
            df_source = pd.DataFrame(data)

            assert df_source.shape[0] > 0, f"The {indicator_id} DataFrame is empty"

            assert (
                abs(df_source.shape[0] - indicator["n_records"]) <= 0.9 * indicator["n_records"]
            ), f"Shape mismatch: expected {indicator['n_records']} but got {df_source.shape[0]}, which exceeds the 90% threshold."

            df = df_source.copy()

            df_indicator = transform_indicator(indicator["id"], df, df_classif1, df_classif2, ISO_3_MAP)

            await publish_series(indicator_id, df_indicator, source_folder=SOURCE_NAME)

    except Exception as e:
        return indicator_id, e


async def process_all_indicators(
    df_annual_indicators, df_classif1, df_classif2, ISO_3_MAP
):

    semaphore = asyncio.Semaphore(MAX_CONCURRENCY)
    failed_indicators = []

    indicators = df_annual_indicators.to_dict(orient="records")

    tasks = [
        process_indicator(semaphore, indicator, df_classif1, df_classif2, ISO_3_MAP)
        for indicator in indicators
    ]

    for future in tqdm_asyncio.as_completed(tasks):
        result = await future
        if isinstance(result, tuple) and len(result) == 2:
            print(result)
            failed_indicators.append(result) 
    return failed_indicators

In [8]:
failed_indicators = await process_all_indicators(
    df_annual_indicators, df_classif1, df_classif2, ISO_3_MAP
)

  0%|          | 0/1063 [00:00<?, ?it/s]/Users/mykhailoslukvin/repo/dv-data-pipeline/dfpp/transformation/source_notebooks/ilo_org/transform.py:211: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_source_filtered["source"] = SOURCE_NAME
/Users/mykhailoslukvin/repo/dv-data-pipeline/dfpp/transformation/source_notebooks/ilo_org/transform.py:212: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_source_filtered["series_id"] = series_id
  0%|          | 4/1063 [03:45<10:58:41, 37.32s/it] 

In [10]:
assert len(failed_indicators) == 0, print(failed_indicators)